# PhaseObjC - Solution Phase Example 1
## Feldspar - number of endmember species equivalent to number of endmember components

Required Python code to load the phase library.

In [ ]:
from ctypes import cdll
from ctypes import util
from rubicon.objc import ObjCClass, objc_method
cdll.LoadLibrary(util.find_library('phaseobjc'))

### Create a Python reference to the `Feldspar` solution phase class, and instantiate an instance of that class.  


In [ ]:
Feldspar = ObjCClass('FeldsparBerman')
obj = Feldspar.alloc().init()

### Obtain properties of the phase inherited from the PhaseBase class.  

In [ ]:
print (obj.phaseName)

## Solution Protocol Functions

### Solution component and species number:
➜ Retrieves the number of endmember components in the system
```
(NSUInteger)numberOfSolutionComponents  
```
➜ Retrieves the number of species (dependent endmembers with positive mole fractions) in the solution
```
(NSUInteger)numberOfSolutionSpecies
```
Note that the number of components (nc) may be the same as the number of endmember species (ns) if the solution does not involve speciation (complexing) or if the solid solution is not a reciprocal solution.

In [ ]:
nc = obj.numberOfSolutionComponents()
print ('Number of components = ', nc)
ns = obj.numberOfSolutionSpecies()
print ('Number of species = ', ns)

### Information about solution components:
Return name, formula, and molecular weight of each endmember component in the solution.  Note the use of the `PhaseBase` class (For further information, see the Stoichiometric Phase notebook examples).  

➜ Retrieves superclass instance of `PhaseBase` object for component at specified index  
```
(id)componentAtIndex:(NSUInteger)index  
```

In [ ]:
PhaseBase = ObjCClass('PhaseBase')
print ("{0:>20s} {1:>20s} {2:>15s}".format('component name', 'formula', 'MW (g/mol)'))
for i in range(0, nc):
    component = obj.componentAtIndex_(i)
    print ("{0:>20s} {1:>20s} {2:15.3f}".format(component.phaseName, component.phaseFormula, component.mw))

### Create a vector of moles of endmember components.
Allocate a "c"-type pointer to a double precision one-dimensional array, and initialize the array to hold the moles of each component in the solution.

In [ ]:
import ctypes
m = (ctypes.c_double*nc)()
ctypes.cast(m, ctypes.POINTER(ctypes.c_double))
m[0] = 1.0
m[1] = 2.0
m[2] = 3.0
for i in range (0, nc):
    component = obj.componentAtIndex_(i)
    print ('moles of (', component.phaseName, ') = ', m[i])

### Note that moles can be assigned from a vector of element abundances using the following functions:
➜ Moles of elements (standard order) => Moles of endmember components of the phase
```
(DoubleVector *)convertElementsToMoles:(double *)e
```
➜ Moles of elements (standard order) => Total moles of endmember components of the phase
```
(double)convertElementsToTotalMoles:(double *)e
```
➜ Moles of elements (standard order) => Total mass of the phase (g)
```
(double)convertElementsToTotalMass:(double *)e
```

In [ ]:
e = (ctypes.c_double*107)()
ctypes.cast(e, ctypes.POINTER(ctypes.c_double))
for i in range (0, 107):
    e[i] = 0.0
e[8]  = m[0]*8.0 + m[1]*8.0 + m[2]*8.0 # O
e[11] = m[0]                           # Na
e[13] = m[0] + 2.0*m[1] + m[2]         # Al
e[14] = m[0]*3.0 + m[1]*2.0 + m[2]*3.0 # Si
e[19] = m[2]                           # K
e[20] = m[1]                           # Ca
mCompute = obj.convertElementsToMoles_(e)
for i in range (0, nc):
    component = obj.componentAtIndex_(i)
    print ('assumed moles of {0:<10s} = {1:5.1f}  computed = {2:5.1f}'.format(component.phaseName, m[i], 
                                                                              mCompute.valueAtIndex_(i)))
print ('Computed total number of moles = ', obj.convertElementsToTotalMoles_(e))
print ('Computed total mass = ', obj.convertElementsToTotalMass_(e))

### Test the mole vector and output derived quantities:
➜ Moles of endmember components => validity of input values
```
(BOOL)testPermissibleValuesOfComponents:(double *)m
```
➜ Moles of endmember components => Moles of elements (standard order)
```
(DoubleVector *)convertMolesToElements:(double *)m
```
➜ Moles of endmember components => Molar sum
```
(double)totalMolesFromMolesOfComponents:(double *)m
```
➜ Moles of endmember components => Mole fractions of endmember components
```
(DoubleVector *)convertMolesToMoleFractions:(double *)m
```

In [ ]:
if (obj.testPermissibleValuesOfComponents_(m) == 1):
    print ('mole input is feasible')
else:
    print ('mole input is infeasible')
    
print ('Total moles = ', obj.totalMolesFromMolesOfComponents_(m))

mole_frac_pointer = obj.convertMolesToMoleFractions_(m)
print ('{0:<20s}{1:<20s}{2:>20s}'.format('component name', 'component formula', 'mole fraction'))
for i in range (0, nc):
    print ('{0:<20s}{1:<20s}{2:20.13e}'.format(obj.componentAtIndex_(i).phaseName, 
                                             obj.componentAtIndex_(i).phaseFormula, 
                                             mole_frac_pointer.valueAtIndex_(i)))

moles_pointer = obj.convertMolesToElements_(m)
ne = moles_pointer.size
formula = ''
for i in range(0, ne):
    value = moles_pointer.valueAtIndex_(i)
    if value != 0.0:
        name = PhaseBase.elementNameFromAtomicNumber_(i)
        formula = formula + name + '(' + str(value) + ')'
print ('Solution formula = ', formula)

### Compute activities and chemical potentials of endmember components:
➜ Moles of components, T (K), P (bars) => activities of endmember components
```
(DoubleVector *)getActivityFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Moles of components, T (K), P (bars) => chemical potentials of endmember components (J)
```
(DoubleVector *)getChemicalPotentialFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```

In [ ]:
t = 1000.0
p = 1000.0
activity = obj.getActivityFromMolesOfComponents_andT_andP_(m, t, p)
potential = obj.getChemicalPotentialFromMolesOfComponents_andT_andP_(m, t, p)
print ('{0:<10s} {1:>20s} {2:>20s}'.format('component', 'activity', 'chemical potential'))
for i in range (0, nc):
    component = obj.componentAtIndex_(i)
    print ('{0:<10s} {1:20.13e} {2:20.13e}'.format(component.phaseName, 
                                               activity.valueAtIndex_(i), 
                                               potential.valueAtIndex_(i)))

### Gibbs free energy and its compositional derivatives:
➜ Moles of components, T (K), P (bars) => Gibbs free energy (J)
```
(double)getGibbsFreeEnergyFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Moles of components, T (K), P (bars) => d(Gibbs free energy)/d(Moles of components) (J)
```
(DoubleVector *)getDgDmFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Moles of components, T (K), P (bars) => d^2(Gibbs free energy)/d(Moles of components)^2 (J)
```
(DoubleMatrix *)getD2gDm2FromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Moles of components, T (K), P (bars) => d^3(Gibbs free energy)/d(Moles of components)^3 (J)
```
(DoubleTensor *)getD3gDm3FromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```

In [ ]:
print ('Gibbs free energy (J) = ', obj.getGibbsFreeEnergyFromMolesOfComponents_andT_andP_(m, t, p))
print ("")
dgdm = obj.getDgDmFromMolesOfComponents_andT_andP_(m, t, p)
for i in range (0, nc):
    print ('dg/dm (', i, ') = ', dgdm.valueAtIndex_(i))
print ("")
d2gdm2 = obj.getD2gDm2FromMolesOfComponents_andT_andP_(m, t, p)
for i in range (0, nc):
    for j in range (0, nc):
        print ('d2g/dm2 (', i, ') (', j, ') = ', d2gdm2.valueAtRowIndex_andColIndex_(i, j))
print ("")
d3gdm3 = obj.getD3gDm3FromMolesOfComponents_andT_andP_(m, t, p)
for i in range (0, nc):
    for j in range (0, nc):
        for k in range (0, nc):
            print ('d3g/dm3 (', i, ') (', j, ') (', k, ') = ', d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(i, j, k))

### Molar derivatives of activities:
➜ Moles of components, T (K), P (bars) => d(activities of endmember components)/d(Moles of components)
```
(DoubleMatrix *)getDaDmFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```

In [ ]:
dadm = obj.getDaDmFromMolesOfComponents_andT_andP_(m, t, p)
for i in range (0, nc):
    for j in range (0, nc):
        print ('da/dm (', i, ') (', j, ') = ', dadm.valueAtRowIndex_andColIndex_(i, j))

### Enthalpy, Entropy, and molar derivatives:
➜ Moles of components, T (K), P (bars) => enthalpy (J)
```
(double)getEnthalpyFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Moles of components, T (K), P (bars) => entropy (J/K)
```
(double)getEntropyFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Moles of components, T (K), P (bars) => d(entropy)/d(Moles of components) (J/K)
```
(DoubleVector *)getDsDmFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Moles of components, T (K), P (bars) => d^2(entropy)/d(Moles of components)^2 (J/K)
```
(DoubleMatrix *)getD2sDm2FromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```

In [ ]:
print ('Enthalpy (J) = ', obj.getEnthalpyFromMolesOfComponents_andT_andP_(m, t, p))
print ('Entropy (J/K) = ', obj.getEntropyFromMolesOfComponents_andT_andP_(m, t, p))
print ("")
dsdm = obj.getDsDmFromMolesOfComponents_andT_andP_(m, t, p)
for i in range (0, nc):
    print ('ds/dm (', i, ') = ', dsdm.valueAtIndex_(i))
print ("")
d2sdm2 = obj.getD2sDm2FromMolesOfComponents_andT_andP_(m, t, p)
for i in range (0, nc):
    for j in range (0, nc):
        print ('d2s/dm2 (', i, ') (', j, ') = ', d2sdm2.valueAtRowIndex_andColIndex_(i, j))

### Heat capacity and its derivatives:
➜ Moles of components, T (K), P (bars) => isobaric heat capacity (J/K)
```
(double)getHeatCapacityFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Moles of components, T (K), P (bars) => d(isobaric heat capacity)/dT (J/K^2)
```
(double)getDcpDtFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Moles of components, T (K), P (bars) => d(isobaric heat capacity)/d(Moles of components) (J/K)
```
(DoubleVector *)getDCpDmFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```

In [ ]:
print ('Heat capacity (J/K) = ', obj.getHeatCapacityFromMolesOfComponents_andT_andP_(m, t, p))
print ("")
print ('dcpdt (J/K^2) = ', obj.getDcpDtFromMolesOfComponents_andT_andP_(m, t, p))
print ("")
dcpdm = obj.getDCpDmFromMolesOfComponents_andT_andP_(m, t, p)
for i in range (0, nc):
    print ('dcp/dm (', i, ') = ', dcpdm.valueAtIndex_(i))

### Volume and its derivatives:
➜ Moles of components, T (K), P (bars) => volume (J/bar)
```
(double)getVolumeFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Moles of components, T (K), P (bars) => d(volume)/d(Moles of components) (J/bar)
```
(DoubleVector *)getDvDmFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Moles of components, T (K), P (bars) => d^2(volume)/d(Moles of components)^2 (J/bar)
```
(DoubleMatrix *)getD2vDm2FromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Moles of components, T (K), P (bars) => d(volume)/dT (J/bar-K)
```
(double)getDvDtFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Moles of components, T (K), P (bars) => d(volume)/dP (J/bar^2)
```
(double)getDvDpFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Moles of components, T (K), P (bars) => d2(volume)/dT^2 (J/bar-K^2)
```
(double)getD2vDt2FromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Moles of components, T (K), P (bars) => d2(volume)/dTdP (J/bar^2-K)
```
(double)getD2vDtDpFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Moles of components, T (K), P (bars) => d2(volume)/dP^2 (J/bar^3)
```
(double)getD2vDp2FromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Moles of components, T (K), P (bars) => d2(volume)/d(Moles of components)dT (J/bar-K)
```
(DoubleVector *)getD2vDmDtFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Moles of components, T (K), P (bars) => d2(volume)/d(Moles of components)dP (J/bar^2)
```
(DoubleVector *)getD2vDmDpFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```

In [ ]:
print ('{0:<10s}{1:13.6f} {2:<15s}'.format('Volume', obj.getVolumeFromMolesOfComponents_andT_andP_(m, t, p), 'J/bar'))
print ('{0:<10s}{1:13.6e} {2:<15s}'.format('dvdt', obj.getDvDtFromMolesOfComponents_andT_andP_(m, t, p), 'J/bar-K'))
print ('{0:<10s}{1:13.6e} {2:<15s}'.format('dvdp', obj.getDvDpFromMolesOfComponents_andT_andP_(m, t, p), 'J/bar^2'))
print ('{0:<10s}{1:13.6e} {2:<15s}'.format('d2vdt2', obj.getD2vDt2FromMolesOfComponents_andT_andP_(m, t, p), 'J/bar-K^2'))
print ('{0:<10s}{1:13.6e} {2:<15s}'.format('d2vdtdp', obj.getD2vDtDpFromMolesOfComponents_andT_andP_(m, t, p),  'J/bar^2-K'))
print ('{0:<10s}{1:13.6e} {2:<15s}'.format('d2vdp2', obj.getD2vDp2FromMolesOfComponents_andT_andP_(m, t, p), 'J/bar^3'))
print ("")
dvdm = obj.getDvDmFromMolesOfComponents_andT_andP_(m, t, p)
for i in range (0, nc):
    print ('dv/dm (', i, ') = ', dvdm.valueAtIndex_(i))
print ("")
d2vdm2 = obj.getD2vDm2FromMolesOfComponents_andT_andP_(m, t, p)
for i in range (0, nc):
    for j in range (0, nc):
        print ('d2v/dm2 (', i, ') (', j, ') = ', d2vdm2.valueAtRowIndex_andColIndex_(i, j))
print ("")
d2vdmdt = obj.getD2vDmDtFromMolesOfComponents_andT_andP_(m, t, p)
for i in range (0, nc):
    print ('d2vdmdt (', i, ') = ', d2vdmdt.valueAtIndex_(i))
print ("")
d2vdmdp = obj.getD2vDmDpFromMolesOfComponents_andT_andP_(m, t, p)
for i in range (0, nc):
    print ('d2vdmdp (', i, ') = ', d2vdmdp.valueAtIndex_(i))

### Accessing properties of solution species:
➜ Moles of components, T (K), P (bars) => formulae as an NSString object
```
(NSString *)getFormulaFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
➜ Retrieves the name of the solution species at the specified index
```
(NSString *)nameOfSolutionSpeciesAtIndex:(NSUInteger)index
```
➜ Moles of solution species => moles of endmember components
```
(DoubleVector *)convertMolesOfSpeciesToMolesOfComponents:(double *)mSpecies
```
➜ Retrieves an elemental stoichiometry vector for the species at the specified index
```
(DoubleVector *)elementalCompositionOfSpeciesAtIndex:(NSUInteger)index
```
➜ Moles of components, T (K), P (bars) => chemical potentials of solution species (J)
```
(DoubleVector *)chemicalPotentialsOfSpeciesFromMolesOfComponents:(double *)m andT:(double)t andP:(double)p
```
Note that the first nc species are identical to the solution components

In [ ]:
print ('formula = ', obj.getFormulaFromMolesOfComponents_andT_andP_(m, t, p))
muSpecies = obj.chemicalPotentialsOfSpeciesFromMolesOfComponents_andT_andP_(m, t, p)
for i in range (0, ns):
    print ('species = ', obj.nameOfSolutionSpeciesAtIndex_(i))
    elm = obj.elementalCompositionOfSpeciesAtIndex_(i)
    for j in range (0, 107):
        if elm.valueAtIndex_(j) > 0.0:
            print ('   element (', j, ') = ', elm.valueAtIndex_(j))
    print ('   chemical potential = ', muSpecies.valueAtIndex_(i))
mSpecies = (ctypes.c_double*3)()
ctypes.cast(mSpecies, ctypes.POINTER(ctypes.c_double))
mSpecies[0] = 1
mSpecies[1] = 2
mSpecies[2] = 3
mSpToComp = obj.convertMolesOfSpeciesToMolesOfComponents_(mSpecies)
for i in range (0, nc):
    print ('moles of component (', i, ') = ', mSpToComp.valueAtIndex_(i))

## Parameter calibration protocol

In [ ]:
try:
    if obj.supportsParameterCalibration() == 1:
        print ('This phase supports the Calibration protocol')
    np = obj.getNumberOfFreeParameters()
    print ('... there are', np, 'parameters')
    names = obj.getArrayOfNamesOfFreeParameters()
    dgdw = obj.getDgDwFromMolesOfComponents_andT_andP_(m, t, p)
    for i in range (0, np):
        name = names.objectAtIndex_(i)
        value = obj.getValueForParameterName_(name)
        units = obj.getUnitsForParameterName_(name)
        print ('Parameter ', name, 'has value ', value, units, 'dgdw = ', dgdw.valueAtIndex_(i))
        dmudw = obj.getChemicalPotentialDerivativesForParameter_usingMolesOfComponents_andT_andP_(name, m, t, p)
        for j in range (0, nc):
            print ('   dmu (', j, ')dw = ', dmudw.valueAtIndex_(j))
except AttributeError:
    print ('This phase does not implement the parameter calibration protocol')